In [1]:
import cv2
import mediapipe as mp
import numpy as np
from collections import deque
import time
import random
import math

In [2]:
def is_fist(hand_landmarks):
    """ Returns True if the hand appears to be closed (fist). """
    # Get key landmark points
    wrist = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST]
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
    pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]

    # Calculate distances from fingertips to wrist
    dist_index = np.linalg.norm([index_tip.x - wrist.x, index_tip.y - wrist.y])
    dist_middle = np.linalg.norm([middle_tip.x - wrist.x, middle_tip.y - wrist.y])
    dist_ring = np.linalg.norm([ring_tip.x - wrist.x, ring_tip.y - wrist.y])
    dist_pinky = np.linalg.norm([pinky_tip.x - wrist.x, pinky_tip.y - wrist.y])
    dist_thumb = np.linalg.norm([thumb_tip.x - wrist.x, thumb_tip.y - wrist.y])

    # If all fingertips are close to the wrist, it's a fist
    if dist_index < 0.1 and dist_middle < 0.1 and dist_ring < 0.1 and dist_pinky < 0.1 and dist_thumb < 0.15:
        return True
    return False

def is_fingers_touching(hand_landmarks, threshold=0.05):
    """Returns True if the index and thumb fingertips are close to each other"""
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]

    # Calculate Euclidean distance between index and thumb
    distance = np.sqrt((index_tip.x - thumb_tip.x) ** 2 + (index_tip.y - thumb_tip.y) ** 2)

    return distance < threshold  # Return True if fingers are close enough


In [3]:

# Function to check if the trace crosses the box
def is_crossing_box(x1, y1, x2, y2, box):
    """Returns True if a line segment crosses a specific box"""
    box_x1, box_y1, box_x2, box_y2 = box
    if (box_x1 <= x1 <= box_x2 and box_y1 <= y1 <= box_y2) or \
       (box_x1 <= x2 <= box_x2 and box_y1 <= y2 <= box_y2):
        return True
    return False

def is_finger_inside_box(x, y):
    """Returns True if the finger is currently inside the box"""
    return box_x1 <= x <= box_x2 and box_y1 <= y <= box_y2
# Function to apply exponential smoothing

# Smoothing Parameters
SMOOTHING_FACTOR = 0.2  # Higher = smoother, but slight lag
def smooth_coordinates(prev, current, alpha=SMOOTHING_FACTOR):
    if prev is None:
        return current
    return alpha * current + (1 - alpha) * prev  # Weighted average of past and current



In [4]:
# Track the number of detected circles
circle_count = 0
last_circle_time = 0  # Prevents rapid consecutive detections
circle_reset_time = 10  # Reset count if no circle detected within 5 seconds

def detect_circle(trail):
    """Detects a circular motion in the finger trail three times before activation"""
    global circle_count, last_circle_time

    if len(trail) < 10:  # Need enough points for a shape
        return False

    # Compute the centroid of the trail
    xs, ys = zip(*[(p[0], p[1]) for p in trail])
    centroid_x, centroid_y = sum(xs) / len(xs), sum(ys) / len(ys)

    # Compute average radius
    distances = [math.sqrt((x - centroid_x) ** 2 + (y - centroid_y) ** 2) for x, y in zip(xs, ys)]
    avg_radius = sum(distances) / len(distances)

    # Check if all points are roughly the same distance from the center (circle-like)
    for d in distances:
        if abs(d - avg_radius) > 10:  # Allow small variation
            return False

    # Get current time
    current_time = time.time()

    # Reset count if no circles detected for `circle_reset_time` seconds
    if current_time - last_circle_time > circle_reset_time:
        circle_count = 0

    # Ensure time gap between circle detections to prevent spamming
    if current_time - last_circle_time > 1:  # Minimum 1 second between circles
        circle_count += 1
        last_circle_time = current_time
        print(f"CIRCLE COUNT: {circle_count}")  # Debugging output

    # Activate special attack only after 3 full circles
    if circle_count >= 3:
        circle_count = 0  # Reset count
        return True

    return False


In [5]:
# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

# Store past positions (smoothing)
history_length = 1  # seconds
fps = 30  # Approximate webcam FPS
max_points = history_length * fps  # Number of frames to store
finger_trails = deque(maxlen=max_points)

# Open webcam
cap = cv2.VideoCapture(0)

# Create a blank whiteboard
WHITEBOARD_SIZE = (600, 800, 3)  # (Height, Width, Channels)
whiteboard = np.ones(WHITEBOARD_SIZE, dtype=np.uint8) * 255  # White background

# Initialize last smoothed position
last_x, last_y = None, None
tracking_enabled = True  # Start with tracking enabled

# Define a grid-aligned box layout
num_rows = 4  # Number of rows
num_cols = 5  # Number of columns
box_size = 40  # Size of each box

# Generate boxes in a grid
boxes = []
start_x, start_y = 100, 100  # Starting position for grid
spacing = 10  # Spacing between boxes

for row in range(num_rows):
    for col in range(num_cols):
        x1 = start_x + col * (box_size + spacing)
        y1 = start_y + row * (box_size + spacing)
        x2, y2 = x1 + box_size, y1 + box_size
        boxes.append((x1, y1, x2, y2))
        
# Create a list to track which boxes are visible (True = visible, False = hidden)
boxes_visible = [True] * len(boxes)
score = 0  
# Track time when a box is destroyed
box_destroy_times = [None] * len(boxes)
special_attack_ready = False  # Special attack is not ready initially
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to RGB (for MediaPipe)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    
    # Clone the whiteboard to refresh the display
    display_canvas = whiteboard.copy()
    # Draw the boxes that are still visible
    for i, box in enumerate(boxes):
        if boxes_visible[i]:  # Draw only if the box is visible
            cv2.rectangle(display_canvas, (box[0], box[1]), (box[2], box[3]), (0, 0, 0), -1)  # Black box
    # If hands detected
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Check if the thumb is folded (toggle tracking)
            if is_fist(hand_landmarks):
                tracking_enabled = False  # Disable tracking when thumb is folded
                finger_trails.clear()  # Clear trails
                cv2.putText(display_canvas, "Tracking Disabled (Fist)", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                continue  # Skip processing hand movement
            
            # If thumb is not folded, enable tracking
            tracking_enabled = True

            # Get index finger tip coordinates
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            h, w, _ = frame.shape
            x, y = int(index_finger_tip.x * WHITEBOARD_SIZE[1]), int(index_finger_tip.y * WHITEBOARD_SIZE[0])

            # Apply smoothing
            x = smooth_coordinates(last_x, x)
            y = smooth_coordinates(last_y, y)
            last_x, last_y = x, y  # Update last known position

            # Store new position in deque
            finger_trails.append((int(x), int(y), time.time()))
            
            if tracking_enabled:
                for i in range(1, len(finger_trails)):
                    x1, y1, _ = finger_trails[i - 1]
                    x2, y2, _ = finger_trails[i]

                    # Check if fingers are touching
                    fingers_touching = False
                    for hand_landmarks in results.multi_hand_landmarks:
                        if is_fingers_touching(hand_landmarks):
                            fingers_touching = True
                            break  # Stop checking after detecting a touch

                    # Check if the box should disappear (fingers touching + trace crossing)
                    if fingers_touching:
                        for j, box in enumerate(boxes):
                            if boxes_visible[j] and is_crossing_box(x1, y1, x2, y2, box):
                                boxes_visible[j] = False  # Hide only this box
                                score += 1  # Increment score
                                box_destroy_times[j] = time.time()  # Record destruction time

                # Restore boxes after 2 seconds
                current_time = time.time()
                for j in range(len(boxes)):
                    if box_destroy_times[j] is not None:  # Ensure we are tracking a destroyed box
                        if current_time - box_destroy_times[j] > 2:  # Check if 2 seconds have passed
                            boxes_visible[j] = True  # Bring the box back
                            box_destroy_times[j] = None  # Reset timer for this box

                # Display the score persistently
                cv2.putText(display_canvas, f"Score: {score}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                # Activate special attack when score reaches 100
                
    if score >= 10:
        special_attack_ready = True
        cv2.putText(display_canvas, "Special Attack Available!", (200, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    # If special attack is ready and a circle is detected, destroy all boxes
    if special_attack_ready and detect_circle(finger_trails):
        boxes_visible = [False] * len(boxes)
        print("SPECIAL ATTACK ACTIVATED!")
        score += len(boxes)  # Increase score for destroying all at once
        special_attack_ready = False  # Reset special attack
    elif not special_attack_ready and detect_circle(finger_trails):
        print("CIRCLE DETECTED SPECIAL NOT READY!")

    # Remove old points (older than 2 seconds)
    current_time = time.time()
    finger_trails = deque([(x, y, t) for x, y, t in finger_trails if current_time - t < history_length], maxlen=max_points)

    # Draw trails only if tracking is enabled
    if tracking_enabled:
        for i in range(1, len(finger_trails)):
            cv2.line(display_canvas, finger_trails[i - 1][:2], finger_trails[i][:2], (0, 0, 0), 2)

    # Show the whiteboard instead of the webcam feed
    cv2.imshow("Virtual Whiteboard Drawing", display_canvas)

    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
